# 要約 
このノートブックは、Hugging FaceのTransformersライブラリを使用して、テキスト分類モデルのロード、前処理、およびトレーニングに関する包括的なガイドを提供しています。具体的には、人間による好み予測に関連するデータセットを扱い、プロンプトに対する応答モデルのパフォーマンスを評価します。以下にノートブックの各セクションの概要を示します。

1. **データのロードとサンプリング**:
   - CSVファイルからデータを読み込み、全体のデータセットの10%をランダムにサンプリングします。これにより、計算リソースを効率的に使いながら処理を迅速化します。

2. **データの分割**:
   - サンプリングしたデータをトレーニングセットとバリデーションセットに分割し、モデル性能の評価を可能にします。

3. **トークナイザーの準備**:
   - 小型のRobertaモデルのトークナイザーを使用して、テキストデータをトークン化します。この処理には、パディング、切り捨て、およびPyTorchテンソル形式での戻り値が含まれています。

4. **モデル設定とトレーニング**:
   - モデルの設定を行い、トレーニング引数を定義した後、トレーニングを開始します。デバイスは、利用可能な場合はGPUを使用します。

5. **推論関数の定義**:
   - トレーニングされたモデルを使用して、トークン化されたデータに対して推論を行う関数を定義します。これにより、ノートブックは推論時のメモリ管理を最適化します。

6. **評価と結果の保存**:
   - テストデータをトークン化し、推論を行った結果をCSVファイルに保存します。最終的に、モデルのロジット結果とデータフレームを結合し、提出用のファイルを作成します。

このノートブックは、データサンプリング、トークン化、モデル設定、推論、および結果保存という体系的なアプローチを通じて、テキスト分類モデルのトレーニングの流れを示しています。特に、計算リソースの効率的な使用を保ちながらモデルのパフォーマンスを維持することに重点が置かれています。使用されている主要なライブラリは、Pandas、Scikit-learn、およびHugging Face Transformersです。

---


# 用語概説 
以下に、Jupyterノートブックで使用されている専門用語や概念の簡単な解説を示します。特に初心者がつまずきそうなものやこのノートブック特有のドメイン知識に焦点を当てています。

1. **トークナイザー**:
   - テキストデータをモデルが理解できる形に変換するためのツールです。具体的には文字や単語を「トークン」と呼ばれる単位に分割し、数値的な表現に変換します。

2. **ロバート (RoBERTa)**:
   - BERT（Bidirectional Encoder Representations from Transformers）を基にしたモデルで、自然言語処理タスクにおける性能を向上させるために、より多くのデータと計算資源を使用してトレーニングされています。

3. **エンコーディング**:
   - トークン化されたデータに対して、モデルが理解できる数値表現（通常はベクトル）の形式に変換するプロセスです。これは、トークンに対応する数値を用意することを含みます。

4. **バッチサイズ**:
   - モデルが一度に処理するデータのサンプル数を指します。大きすぎるとメモリが不足し、小さすぎると効率が悪くなります。

5. **ウォームアップステップ**:
   - 学習の初期段階で学習率を徐々に上げていく手法です。これにより、最初のエポックでの急激な変化による不安定さを避けます。

6. **トレーニング引数 (Training Arguments)**:
   - モデルトレーニング時に指定する設定やハイパーパラメータのことです。これにはエポック数、バッチサイズ、ロギングの頻度などが含まれます。

7. **ロジット (Logits)**:
   - モデルの出力で、各クラスに対するスコアを示す値です。これをsoftmax関数に通すことで、確率に変換されます。ロジットが高いほど、そのクラスが選ばれる可能性が高いとなります。

8. **アテンションマスク (Attention Mask)**:
   - トークン化されたデータの中で、どのトークンが有効か（例えば、パディングしたトークンを無視するためのマスク）を示すバイナリマスクです。1は有効、0は無効を表します。

9. **カスタムデータセットクラス**:
   - PyTorchのデータセットクラスを継承して作成したクラスで、トレーニングデータとラベルを格納し、データを効率よく取得できるようにカスタマイズされています。

これらの用語や概念は、特に実務経験がない初心者にとって馴染みが少ないかもしれませんが、機械学習プロジェクトを実施する際には重要です。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Introduction
 This notebook provides a comprehensive guide for loading, preprocessing, and training a text classification model using the Hugging Face Transformers library. We will use the Roberta tokenizer and a smaller Roberta model to manage computational resources efficiently. The process includes data loading, sampling, tokenization, and model preparation.

# Step 1: Load and Sample Data
 First, we load the dataset from a CSV file and take a random sample to reduce the dataset size for quicker processing.


</div>
<div class="column-right">

# 日本語訳

# はじめに
このノートブックは、Hugging Face Transformersライブラリを使用してテキスト分類モデルのロード、前処理、トレーニングに関する包括的なガイドを提供します。計算リソースを効率的に管理するために、ロバートトークナイザーと小型のロバートモデルを使用します。プロセスには、データのロード、サンプリング、トークナイゼーション、およびモデルの準備が含まれます。

# ステップ 1: データのロードとサンプリング
最初に、CSVファイルからデータセットをロードし、処理を迅速化するためにデータセットのサイズを減少させるためにランダムサンプルを取得します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

# Sample 10% of the data
df_sample = df.sample(frac=0.1, random_state=42)

# Check sample data
print(df_sample.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd
from sklearn.model_selection import train_test_split

# データをロードする
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

# データの10%をサンプリングする
df_sample = df.sample(frac=0.1, random_state=42)

# サンプルデータを確認する
print(df_sample.head())  # サンプルデータの最初の5行を表示します。
```

</div>
</details>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データをロードする
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")

# データの10%をサンプリングする
df_sample = df.sample(frac=0.1, random_state=42)

# サンプルデータを確認する
print(df_sample.head())  # サンプルデータの最初の5行を表示します。

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Step 2: Split Data into Training and Validation Sets
We split the data into training and validation sets to evaluate the model's performance.

</div>
<div class="column-right">

# 日本語訳

# ステップ 2: データをトレーニングセットとバリデーションセットに分割する
データをトレーニングセットとバリデーションセットに分割し、モデルのパフォーマンスを評価します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sample['prompt'].tolist(), 
    df_sample['winner_model_a'], 
    test_size=0.1, 
    random_state=42
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データをトレーニングセットとバリデーションセットに分割する
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sample['prompt'].tolist(),  # プロンプト（入力テキスト）のリストを取得します
    df_sample['winner_model_a'],    # モデルAの勝者ラベルを取得します
    test_size=0.1,                  # データの10%をテストセットとします
    random_state=42                 # 再現性のために乱数シードを設定します
)
```

</div>
</details>

In [ ]:
# データをトレーニングセットとバリデーションセットに分割する
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_sample['prompt'].tolist(),  # プロンプト（入力テキスト）のリストを取得します
    df_sample['winner_model_a'],    # モデルAの勝者ラベルを取得します
    test_size=0.1,                  # データの10%をテストセットとします
    random_state=42                 # 再現性のために乱数シードを設定します
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Step 3: Load and Prepare the Tokenizer
We use a smaller Roberta model tokenizer for efficiency and tokenize the text data.

</div>
<div class="column-right">

# 日本語訳

# ステップ 3: トークナイザーをロードして準備する
効率のために小型のロバートモデルのトークナイザーを使用し、テキストデータをトークン化します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import RobertaTokenizer

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize function
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize texts
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# Check tokenized data
print(train_encodings)
print(val_encodings)
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import RobertaTokenizer

# トークナイザーをロードする
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")  # ロバートベースモデルのトークナイザーを取得します

# トークン化関数
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding="max_length",  # トークンのパディングを最大長に設定します
        truncation=True,        # テキストが最大長を超えた場合に切り捨てます
        max_length=128,        # 最大トークン数を128に設定します
        return_tensors='pt'    # PyTorchテンソルとして戻します
    )

# テキストをトークン化する
train_encodings = tokenize_function(train_texts)  # トレーニングデータをトークン化します
val_encodings = tokenize_function(val_texts)      # バリデーションデータをトークン化します

# トークン化されたデータを確認する
print(train_encodings)  # トークン化されたトレーニングデータを表示します
print(val_encodings)    # トークン化されたバリデーションデータを表示します
```

</div>
</details>

In [ ]:
from transformers import RobertaTokenizer

# トークナイザーをロードする
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")  # ロバートベースモデルのトークナイザーを取得します

# トークン化関数
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding="max_length",  # トークンのパディングを最大長に設定します
        truncation=True,        # テキストが最大長を超えた場合に切り捨てます
        max_length=128,        # 最大トークン数を128に設定します
        return_tensors='pt'    # PyTorchテンソルとして戻します
    )

# テキストをトークン化する
train_encodings = tokenize_function(train_texts)  # トレーニングデータをトークン化します
val_encodings = tokenize_function(val_texts)      # バリデーションデータをトークン化します

# トークン化されたデータを確認する
print(train_encodings)  # トークン化されたトレーニングデータを表示します
print(val_encodings)    # トークン化されたバリデーションデータを表示します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Step 4: Prepare the Model Configuration
 Set up the configuration for the model, ensuring it uses the appropriate device (GPU if available).

</div>
<div class="column-right">

# 日本語訳

# ステップ 4: モデルの設定を準備する
モデルの設定を行い、適切なデバイス（利用可能な場合はGPU）を使用するようにします。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import torch
import numpy as np

# Config class
class Config:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.batch_size = 16

cfg = Config()

# Load model
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(cfg.device)

# CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = np.array(labels, dtype=int)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

# TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=cfg.batch_size,
    per_device_eval_batch_size=cfg.batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train model
trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import torch
import numpy as np

# 設定クラス
class Config:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPUが使用可能であればGPUを、そうでなければCPUを使用します
        self.batch_size = 16  # バッチサイズを16に設定します

cfg = Config()

# モデルをロードする
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(cfg.device)  # ロバートベースモデルを2つのラベルで分類するためにロードし、指定したデバイスに移動します

# カスタムデータセットクラス
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # エンコーディングを格納します
        self.labels = np.array(labels, dtype=int)  # ラベルを整数のNumPy配列として格納します
    
    def __getitem__(self, idx):
        # 指定したインデックスのエンコーディングとラベルを取得します
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}  # エンコーディングをテンソル化します
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # ラベルをテンソル化します
        return item
    
    def __len__(self):
        return len(self.labels)  # データセットの長さを返します

# データセットを作成する
train_dataset = CustomDataset(train_encodings, train_labels)  # トレーニングデータセットを作成します
val_dataset = CustomDataset(val_encodings, val_labels)        # バリデーションデータセットを作成します

# トレーニング引数
training_args = TrainingArguments(
    output_dir='./results',                # 結果を保存するディレクトリ
    num_train_epochs=3,                    # トレーニングを行うエポック数
    per_device_train_batch_size=cfg.batch_size,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=cfg.batch_size,   # デバイスごとの評価バッチサイズ
    warmup_steps=500,                       # ウォームアップステップ数
    weight_decay=0.01,                     # 重み減衰率
    logging_dir='./logs',                   # ログを保存するディレクトリ
    logging_steps=10,                       # ロギングのステップ頻度
)

# トレーナーを初期化する
trainer = Trainer(
    model=model,                            # モデル
    args=training_args,                     # トレーニング引数
    train_dataset=train_dataset,            # トレーニングデータセット
    eval_dataset=val_dataset                 # バリデーションデータセット
)

# モデルをトレーニングする
trainer.train()  # モデルのトレーニングを開始します
```

</div>
</details>

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import torch
import numpy as np

# 設定クラス
class Config:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPUが使用可能であればGPUを、そうでなければCPUを使用します
        self.batch_size = 16  # バッチサイズを16に設定します

cfg = Config()

# モデルをロードする
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2).to(cfg.device)  # ロバートベースモデルを2つのラベルで分類するためにロードし、指定したデバイスに移動します

# カスタムデータセットクラス
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # エンコーディングを格納します
        self.labels = np.array(labels, dtype=int)  # ラベルを整数のNumPy配列として格納します
    
    def __getitem__(self, idx):
        # 指定したインデックスのエンコーディングとラベルを取得します
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}  # エンコーディングをテンソル化します
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # ラベルをテンソル化します
        return item
    
    def __len__(self):
        return len(self.labels)  # データセットの長さを返します

# データセットを作成する
train_dataset = CustomDataset(train_encodings, train_labels)  # トレーニングデータセットを作成します
val_dataset = CustomDataset(val_encodings, val_labels)        # バリデーションデータセットを作成します

# トレーニング引数
training_args = TrainingArguments(
    output_dir='./results',                # 結果を保存するディレクトリ
    num_train_epochs=3,                    # トレーニングを行うエポック数
    per_device_train_batch_size=cfg.batch_size,  # デバイスごとのトレーニングバッチサイズ
    per_device_eval_batch_size=cfg.batch_size,   # デバイスごとの評価バッチサイズ
    warmup_steps=500,                       # ウォームアップステップ数
    weight_decay=0.01,                     # 重み減衰率
    logging_dir='./logs',                   # ログを保存するディレクトリ
    logging_steps=10,                       # ロギングのステップ頻度
)

# トレーナーを初期化する
trainer = Trainer(
    model=model,                            # モデル
    args=training_args,                     # トレーニング引数
    train_dataset=train_dataset,            # トレーニングデータセット
    eval_dataset=val_dataset                 # バリデーションデータセット
)

# モデルをトレーニングする
trainer.train()  # モデルのトレーニングを開始します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Step 5: Inference Function
Define a function to perform inference on the tokenized data using the trained model.

</div>
<div class="column-right">

# 日本語訳

# ステップ 5: 推論関数
トレーニングされたモデルを使用してトークン化されたデータに対して推論を行う関数を定義します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def infer(model, input_ids, attention_mask, batch_size=cfg.batch_size):
    model.eval()
    results = []
    with torch.no_grad():
        for i in range(0, len(input_ids), batch_size):
            batch_input_ids = input_ids[i:i + batch_size].to(cfg.device)
            batch_attention_mask = attention_mask[i:i + batch_size].to(cfg.device)
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            results.extend(outputs.logits.cpu().numpy())
    return np.array(results)
```

</div>
<div class="column-right">

# 日本語訳

```python
def infer(model, input_ids, attention_mask, batch_size=cfg.batch_size):
    model.eval()  # モデルを評価モードに設定します
    results = []  # 結果を格納するリストを初期化します
    with torch.no_grad():  # 勾配計算を無効にしてメモリを節約します
        for i in range(0, len(input_ids), batch_size):  # バッチサイズごとにループします
            batch_input_ids = input_ids[i:i + batch_size].to(cfg.device)  # バッチの入力IDをデバイスに移動します
            batch_attention_mask = attention_mask[i:i + batch_size].to(cfg.device)  # バッチのアテンションマスクをデバイスに移動します
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)  # モデルの出力を取得します
            results.extend(outputs.logits.cpu().numpy())  # 出力のロジットをCPUに移動し、NumPy配列に変換して結果に追加します
    return np.array(results)  # 結果をNumPy配列として返します
```

</div>
</details>

In [ ]:
def infer(model, input_ids, attention_mask, batch_size=cfg.batch_size):
    model.eval()  # モデルを評価モードに設定します
    results = []  # 結果を格納するリストを初期化します
    with torch.no_grad():  # 勾配計算を無効にしてメモリを節約します
        for i in range(0, len(input_ids), batch_size):  # バッチサイズごとにループします
            batch_input_ids = input_ids[i:i + batch_size].to(cfg.device)  # バッチの入力IDをデバイスに移動します
            batch_attention_mask = attention_mask[i:i + batch_size].to(cfg.device)  # バッチのアテンションマスクをデバイスに移動します
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)  # モデルの出力を取得します
            results.extend(outputs.logits.cpu().numpy())  # 出力のロジットをCPUに移動し、NumPy配列に変換して結果に追加します
    return np.array(results)  # 結果をNumPy配列として返します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Step 6: Evaluate and Save Results
Tokenize the test data, perform inference, and save the results to a CSV file.

</div>
<div class="column-right">

# 日本語訳

# ステップ 6: 評価と結果の保存
テストデータをトークン化し、推論を行い、結果をCSVファイルに保存します。

</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Tokenize the test data
test_texts = df_sample['prompt'].tolist()
test_encodings = tokenize_function(test_texts)

# Perform inference
results = infer(model, test_encodings['input_ids'], test_encodings['attention_mask'])

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results, columns=['logit_a', 'logit_b'])
submission = pd.concat([df_sample[['id']], results_df], axis=1)
submission.to_csv('submission.csv', index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# テストデータをトークン化する
test_texts = df_sample['prompt'].tolist()  # プロンプトのリストを取得します
test_encodings = tokenize_function(test_texts)  # テストデータをトークン化します

# 推論を行う
results = infer(model, test_encodings['input_ids'], test_encodings['attention_mask'])  # トークン化されたデータに対して推論を実行します

# 結果をDataFrameに変換し、CSVとして保存する
results_df = pd.DataFrame(results, columns=['logit_a', 'logit_b'])  # ロジットの結果をDataFrameに変換します
submission = pd.concat([df_sample[['id']], results_df], axis=1)  # IDと結果を結合します
submission.to_csv('submission.csv', index=False)  # CSVファイルとして保存します
```

</div>
</details>

In [ ]:
# テストデータをトークン化する
test_texts = df_sample['prompt'].tolist()  # プロンプトのリストを取得します
test_encodings = tokenize_function(test_texts)  # テストデータをトークン化します

# 推論を行う
results = infer(model, test_encodings['input_ids'], test_encodings['attention_mask'])  # トークン化されたデータに対して推論を実行します

# 結果をDataFrameに変換し、CSVとして保存する
results_df = pd.DataFrame(results, columns=['logit_a', 'logit_b'])  # ロジットの結果をDataFrameに変換します
submission = pd.concat([df_sample[['id']], results_df], axis=1)  # IDと結果を結合します
submission.to_csv('submission.csv', index=False)  # CSVファイルとして保存します

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Conclusion
This notebook provided a structured approach to loading, preprocessing, and training a text classification model using the Transformers library. The steps included data sampling, tokenization, model configuration, and inference. The final results were saved for further analysis. This methodology ensures efficient use of computational resources while maintaining model performance.

</div>
<div class="column-right">

# 日本語訳

# 結論
このノートブックでは、Transformersライブラリを使用してテキスト分類モデルのロード、前処理、トレーニングに関する体系的なアプローチを提供しました。ステップにはデータのサンプリング、トークン化、モデルの設定、推論が含まれています。最終的な結果はさらなる分析のために保存されました。この方法論は、モデルのパフォーマンスを維持しながら計算リソースの効率的な使用を確保します。

</div>